In [20]:
from ase import io
from ase import Atoms
from ase.visualize import view
from ase import geometry
from ase.io import read
import numpy as np


In [21]:
a= 4.497
b=2*a*np.sin(60*np.pi/180)
print(a,b)

4.497 7.789032481637241


In [22]:
# Load Relaxed, unstrained slab. Nothe the file has been
# previously relaxed with ttm3f

direct='/Users/marivi/RESEARCH/TTM/Periodic-Betul/SRC_periodic/minener_CG/Ice_Slab/6_15_8/Snap0dip/'
file = 'relax.xyz'
slab=read(direct+file) 
Na = len(slab)
slab.cell=[[        26.982000,   0.000000,    0.000000],
        [0.000000 ,   116.835000  ,  0.000000],
        [0.000000  ,  0.000000  , 58.568000 ]]


In [23]:
#slab.get_positions()[0]
view(slab)
#Na = len(slab)

<Popen: returncode: None args: ['/Users/marivi/SOFT/miniforge3/bin/python3.9...>

In [311]:
# Note the non periodic direction is y

#slab.cell=[[        22.485000,   0.000000,    0.000000],
#        [0.000000 ,   77.890000  ,  0.000000],
#        [0.000000  ,  0.000000  , 40.000 ]]



In [5]:
# Create an array with all the OH vectors, in order
slab_o=slab[slab.symbols == 'O'].copy()
slab_h=slab[slab.symbols == 'H'].copy()
slab_vh=[]
for i in range(int(Na/3)):
    vh1=slab_h.get_positions()[2*i]-slab_o.get_positions()[i]
    vh2=slab_h.get_positions()[2*i+1]-slab_o.get_positions()[i]
    slab_vh.append(vh1)
    slab_vh.append(vh2)


In [6]:
# This part of the code is to Apply strain gradient to O slab along Z
# Not useful for the relaxation because all O atoms would have to be constrained
sg=1 # 0.1% strain gradient
minZ=min(slab_o.positions[:][2]) # Minimum z coordinate
slab_os=slab_o.copy()
for i in range(int(Na/3)):
    deltaz = slab_os.positions[i][2]-minZ
    deltaz = deltaz*( 1 + deltaz*sg/100)
    slab_os.positions[i][2]=deltaz
# Create a new Strained slab object from original
slab_str=slab.copy()
for i in range(int(Na/3)):
    slab_str.positions[3*i]=slab_os.positions[i]
    slab_str.positions[3*i+1]=slab_os.positions[i]+slab_vh[2*i]
    slab_str.positions[3*i+2]=slab_os.positions[i]+slab_vh[2*i+1]


In [7]:
view(slab_str)

<Popen: returncode: None args: ['/Users/marivi/SOFT/miniforge3/bin/python3.9...>

In [8]:
slab_str.write("slab_stg.xyz")

In [26]:
# Now repeat but applying a sinusoudal wave.
# I'm applying a half wavelenght deformation (note w=pi/l and not 2pi/l)
# The long distance (wavelength is along the y direction)
# This code will generate multiple files, deforming along z and x each with
# 8 different amplitudes.
# For the relaxation the first layer will need to be constrained. We understand 1st layer as that with smallest z or x
# both have been set to be zero.
# The constraint routine needs to change depending on which direction is kept fixed. But note that the ttmm3f relaxation
# code has been tightly couple to this part, so the code is only usable for this project. Also
# Ones needs to recompile if the strain direction is changes and the constrain
# subroutine needs to be rewritten accordingly.

l = slab.cell[1][1]
w = 2*np.pi/l 

for ia in range(9):

    a = float(ia) # Amplitude of the wave in Ang, i=0 means the unconstrained slab, which we need for the analysis
    minZ=min(slab_o.positions[:][2]) # Minimum z coordinate, will determine the origin.
    minY=min(slab_o.positions[:][1]) # Minimum y coordinate
    minX=min(slab_o.positions[:][0]) # Minimum x coordinate

    slab_os=slab_o.copy()
    # Strain along z
    for i in range(int(Na/3)):
        #Rigid shift of slab
        deltaz = slab_o.positions[i][2]-minZ
        deltay = slab_o.positions[i][1]-minY
        deltax = slab_o.positions[i][0]-minY
        # Apply strain along z
        deltaz = deltaz + a*np.sin(w*deltay)
        slab_os.positions[i][2]=deltaz
        slab_os.positions[i][1]=deltay
    # Create a new Strained slab object from original
    slab_str=slab.copy()
    for i in range(int(Na/3)):
        slab_str.positions[3*i]=slab_os.positions[i]
        slab_str.positions[3*i+1]=slab_os.positions[i]+slab_vh[2*i]
        slab_str.positions[3*i+2]=slab_os.positions[i]+slab_vh[2*i+1]
    slab_str.write(direct+"slab_singz"+str(ia)+".xyz")    
    # Strain along x
    slab_os=slab_o.copy()
    for i in range(int(Na/3)):
        #Rigid shift of slab
        deltaz = slab_o.positions[i][2]-minZ
        deltay = slab_o.positions[i][1]-minY
        deltax = slab_o.positions[i][0]-minY
        # Apply strain along x
        deltax = deltax + a*np.sin(w*deltay)
        slab_os.positions[i][0]=deltax
        slab_os.positions[i][1]=deltay
    # Create a new Strained slab object from original
    slab_str=slab.copy()
    for i in range(int(Na/3)):
        slab_str.positions[3*i]=slab_os.positions[i]
        slab_str.positions[3*i+1]=slab_os.positions[i]+slab_vh[2*i]
        slab_str.positions[3*i+2]=slab_os.positions[i]+slab_vh[2*i+1]
    slab_str.write(direct+"slab_singx"+str(ia)+".xyz")                   


In [27]:
view(slab_str)

<Popen: returncode: None args: ['/Users/marivi/SOFT/miniforge3/bin/python3.9...>

In [24]:
# Apply a Bending Strain Deformation
d = slab.cell[1][1] # The longest slab dimension
h = slab.cell[0][0] # The shortest slab dimension

R = 2*d # This needs to be adjusted
theta = 2*np.arcsin(d/(2*R))

minZ=min(slab_o.positions[:][2]) # Minimum z coordinate, will determine the origin.
minY=min(slab_o.positions[:][1]) # Minimum y coordinate
minX=min(slab_o.positions[:][0]) # Minimum x coordinate

slab_os=slab_o.copy()
    # Bending yz plane
for i in range(int(Na/3)):
        #Rigid shift of slab
    deltaz = slab_o.positions[i][2]-minZ
    deltay = slab_o.positions[i][1]-minY
    deltax = slab_o.positions[i][0]-minY
        # Apply strain along z
    deltax = deltax + R*np.cos(theta/2)-(R-deltax)*np.cos(theta*(deltay/d-0.5))
    deltay = deltay + d/2+(R-deltax)*np.sin(theta*(deltay/d-0.5))
    slab_os.positions[i][2]=deltaz
    slab_os.positions[i][1]=deltay
    slab_os.positions[i][0]=deltax

    # Create a new Strained slab object from original
slab_str=slab.copy()
for i in range(int(Na/3)):
    slab_str.positions[3*i]=slab_os.positions[i]
    slab_str.positions[3*i+1]=slab_os.positions[i]+slab_vh[2*i]
    slab_str.positions[3*i+2]=slab_os.positions[i]+slab_vh[2*i+1]
slab_str.write(direct+"slab_bend"+str(ia)+".xyz")    

In [25]:
view(slab_str)

<Popen: returncode: None args: ['/Users/marivi/SOFT/miniforge3/bin/python3.9...>

In [19]:
print(d,h,R, theta)

116.835 26.982 233.67 0.5053605102841573
